# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Final Heatmap Image
![This should work](heatmap.png)

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [110]:
#imports .csv
weather_data_file = "input/weather_data.csv"
#loads csv into dataframe
weather_df = pd.read_csv(weather_data_file)
weather_df['City']=weather_df['City'].str.title()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [111]:
#configures gmaps with api key stored in seperate file
gmaps.configure(api_key=gkey)
#loading coordinate data and humidity into lists and assigning them type float
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)



In [112]:
#loads figure into variable, zoom level, map center, width and height adjusted for readability
fig = gmaps.figure(zoom_level=2, center=(25,5),layout={'width':'950px', 'height':'600px'})
#create heat layer using humidity and coordinate data
#intensity variable input so the highest humidity will have the maximum intensity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weather_df["Humidity"].max(),
                                 point_radius = 2)
#apply heat layer to map
fig.add_layer(heat_layer)
#outputs map
fig

Figure(layout=FigureLayout(height='600px', width='950px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [113]:
#This section narrows the dataframe down to cities have cloudiness under 50, humidity under 75 
# and temperatures between 0 and 32 degrees fahrenheit
vacation=weather_df.loc[weather_df["Cloudiness"]<50]
vacation=vacation.loc[vacation["Humidity"]<75]
vacation=vacation.loc[vacation["Max Temp in F"]>0]
vacation=vacation.loc[vacation["Max Temp in F"]<32]
vacation=vacation.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [114]:
#load dataframe into requested variable
hotel_df=vacation
#add empty column for hotel name
hotel_df["Hotel Name"]=""

In [115]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000,
        "type": "lodging", #fun fact 'hotel' is not a type in the google API
        "keyword":"hotel",
        "key": gkey,
        }
for index, row in hotel_df.iterrows():

    # get city name from df
    city_name = row['City']
    # add location to params dict
    params['location'] = f"{hotel_df.at[index,'Lat']}, {hotel_df.at[index,'Lng']}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
                
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 43: Birjand.
Closest hotel to Birjand is هتل پرويز.
------------
Retrieving Results for Index 46: Albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 76: Chengde.
Closest hotel to Chengde is Yunshan Hotel.
------------
Retrieving Results for Index 84: Petropavlovsk-Kamchatskiy.
Closest hotel to Petropavlovsk-Kamchatskiy is Hotel Comfort.
------------
Retrieving Results for Index 141: Jiutai.
Closest hotel to Jiutai is Echarm(changchun Jiutai.
------------
Retrieving Results for Index 164: Ilulissat.
Closest hotel to Ilulissat is Hotel Arctic.
------------
Retrieving Results for Index 185: Linxia.
Closest hotel to Linxia is Hehuang Mingzhu International Hotel.
------------
Retrieving Results for Index 206: Mitchell.
Closest hotel to Mitchell is Comfort Inn & Suites.
------------
Retrieving Results for Index 222: Husavik.
Closest hotel to Husavik is Askja Apartment.
------------
Retrieving Results for Index 279: Varhaug.
Missin

In [116]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [117]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig




Figure(layout=FigureLayout(height='600px', width='950px'))